<a href="https://colab.research.google.com/github/agmorcillo/Coursera_Capstone/blob/main/Practice_Week_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Practice Week 3 - Part 1**

### Loading libraries - Part 1

In [23]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np

### Scrapping the url

In [22]:
url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(url, 'lxml')

table = soup.find("table")
table_rows = table.tbody.find_all("tr")



clean = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text for tr in td]
    
    # Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    if row != [] and row[1] != "Not assigned\n":
        # If a cell has a borough but a "Not assigned" neighbourhood, then the neighborhood will be the same as the borough.
        if "Not assigned\n" in row[2]: 
            row[2] = row[1]
        clean.append(row)

# Dataframe with the main columns
df = pd.DataFrame(clean, columns = ["PostalCode", "Borough", "Neighbourhood"])

#Cleaning "/n" from de columns Neighbourhood, Borough and PostalCode

df["Neighbourhood"] = df["Neighbourhood"].str.replace("\n","")
df["Borough"] = df["Borough"].str.replace("\n","")
df["PostalCode"] = df["PostalCode"].str.replace("\n","")

df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


**Group all neighborhoods using postal code**

In [19]:
df = df.groupby(["PostalCode", "Borough"])["Neighbourhood"].apply(", ".join).reset_index()
df.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [21]:
print("Shape: rows and columns", df.shape)

Shape: rows and columns (103, 3)


# **Practice Week 3 - Part 2**

Loading libraries - Part 2

In [29]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [30]:
df_geo_coor = pd.read_csv("sample_data/Geospatial_Coordinates.csv")
df_geo_coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [31]:
# Merge Latitude and Longitude using PostalCode
df_toronto = pd.merge(df, df_geo_coor, how='left', left_on = 'PostalCode', right_on = 'Postal Code')

# delete the PostalCode column
df_toronto.drop("PostalCode", axis=1, inplace=True)
df_toronto.head()

,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,North York,Parkwoods,M3A,43.753259,-79.329656
1,North York,Victoria Village,M4A,43.725882,-79.315572
2,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M7A,43.662301,-79.389494


# **Practice Week 3 - Part 3**

## Loading libraries - Part 2

In [33]:
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe
import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors